# Parte 2 del ejercicio del módulo 5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler, LabelEncoder, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif # Para SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error,root_mean_squared_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR, SVC
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import  f_regression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow import keras

from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import multilabel_confusion_matrix


## Cargar el dataset

Repito algunos pasos del ejercicio de EDAS, porque son claves para poder trabajar.

In [2]:
# Cargar el dataset
file_path = "../Data/airbnb_nyc_clean.csv"  # Ruta del archivo subido
df = pd.read_csv(file_path)

Información del Dataset:
RangeIndex: 69305 entries, 0 to 69304
Data columns (total 23 columns):

dtypes: bool(1), float64(11), int64(2), object(9)
memory usage: 11.7+ MB


In [3]:
# Mostrar las primeras filas del dataset
df.head(1)

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,instant_bookable,...,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,False,...,966.0,193.0,10.0,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...


###  Eliminar columnas id, host_id y host_name

In [ ]:
# Eliminar columnas que no aportan nada de momento
df.drop(columns=['id', 'name', 'host_id', 'host_name', 'house_rules','neighbourhood'], inplace=True, errors='ignore', axis=1)

print(df.shape)

(69305, 17)


### Separar por tipo de columnas en categóricas y numéricas

In [5]:
def marcar_num_columns(df):
    numerical_columns = df.select_dtypes(exclude=['object', 'category']).columns.to_list() 
    print(numerical_columns)
    print(len(numerical_columns))
    return(numerical_columns)
numerical_columns = marcar_num_columns(df)  

['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365']
12


In [6]:
print(numerical_columns)


['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365']


In [7]:
def marcar_cat_columns(df):
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns.to_list()
    print(categorical_columns)
    print(len(categorical_columns))
    return(categorical_columns)
categorical_columns = marcar_cat_columns(df)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type', 'last_review']
5


### Ver los nombres de las columnas

In [8]:
def indices_columnas(dataframe):
    df_index = df.keys()  # obtenemos los nombres de las columnas
    #print(df_index)
    return(df_index)

df_index = indices_columnas(df)

### Ver valores únicos de las columnas

In [9]:
def valores_unicos_all(df):
    # Find number of unique values in the dataset
    for col in df.columns:
        print(f'Column {col} \t has {df[col].nunique()} unique values')
        
valores_unicos_all(df)

Column host_identity_verified 	 has 2 unique values
Column neighbourhood_group 	 has 5 unique values
Column lat 	 has 21998 unique values
Column long 	 has 17780 unique values
Column instant_bookable 	 has 2 unique values
Column cancellation_policy 	 has 3 unique values
Column room_type 	 has 4 unique values
Column construction_year 	 has 20 unique values
Column price 	 has 1152 unique values
Column service_fee 	 has 232 unique values
Column minimum_nights 	 has 14 unique values
Column number_of_reviews 	 has 476 unique values
Column last_review 	 has 2472 unique values
Column reviews_per_month 	 has 1016 unique values
Column review_rate_number 	 has 5 unique values
Column calculated_host_listings_count 	 has 78 unique values
Column availability_365 	 has 437 unique values


In [10]:
marcar_cat_columns(df)

def valores_unicos_cat(df):
    categorical_columns = marcar_cat_columns(df)
    for col in categorical_columns:
        print('________________________________________________________')
        print(f"Valores únicos en la columna '{col}':")
        print('cantidad de valores únicos:', len(df[col].unique()))
        print(f'Los valores únicos de de la columna {col} son:', df[col].unique())
        print()
        print(df[col].unique())
        print(f'usando value_counts: {df[col].value_counts()}')
        print()
    return(categorical_columns)    
        
valores_unicos_cat(df)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type', 'last_review']
5
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type', 'last_review']
5
________________________________________________________
Valores únicos en la columna 'host_identity_verified':
cantidad de valores únicos: 2
Los valores únicos de de la columna host_identity_verified son: ['unconfirmed' 'verified']

['unconfirmed' 'verified']
usando value_counts: host_identity_verified
unconfirmed    34779
verified       34526
Name: count, dtype: int64

________________________________________________________
Valores únicos en la columna 'neighbourhood_group':
cantidad de valores únicos: 5
Los valores únicos de de la columna neighbourhood_group son: ['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']

['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']
usando value_counts: neighbourhood_group
Manhattan        29587
Brooklyn         27895
Queens          

['host_identity_verified',
 'neighbourhood_group',
 'cancellation_policy',
 'room_type',
 'last_review']

### Vale la pena estudiar las categóricas siguientes:

**1. host_identity_verified: ['unconfirmed' 'verified']**

| Categoría    | Conteo |
|--------------|--------|
| unconfirmed  | 34779  |
| verified     | 34526  |

**2. neighbourhood_group: ['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']**

| Categoría     | Conteo |
|---------------|--------|
| Manhattan     | 29587  |
| Brooklyn      | 27895  |
| Queens        | 9262   |
| Bronx         | 1912   |
| Staten Island | 649    |

**3. cancellation_policy: ['strict' 'moderate' 'flexible']**

| Categoría  | Conteo |
|------------|--------|
| moderate   | 23244  |
| flexible   | 23126  |
| strict     | 22935  |

**4. room_type: ['Private room' 'Entire home/apt' 'Shared room' 'Hotel room']**

| Categoría        | Conteo |
|------------------|--------|
| Entire home/apt  | 37212  |
| Private room     | 30508  |
| Shared room      | 1470   |
| Hotel room       | 115    |

### Fecha: probar a descomponer la fecha en campos año mes y día con pandas es decir, en 3 columnas, por ejemplo usando to_datetime de pandas y los accesores de fecha para extraer año mes y día.

In [12]:
# Convertir la columna de fecha `last_review` a tipo datetime
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# Crear nuevas columnas para año, mes y día
df['review_year'] = df['last_review'].dt.year
df['review_month'] = df['last_review'].dt.month
df['review_day'] = df['last_review'].dt.day

# Verificar cambios
print("Nuevas columnas creadas:")
print(df[['last_review', 'review_year', 'review_month', 'review_day']].head())

Nuevas columnas creadas:
  last_review  review_year  review_month  review_day
0  2021-10-19         2021            10          19
1  2022-05-21         2022             5          21
2  2019-06-14         2019             6          14
3  2019-07-05         2019             7           5
4  2018-11-19         2018            11          19


In [13]:
# Eliminar la columna original `last_review` si ya no es necesaria
df.drop(columns=['last_review'], inplace=True, errors='ignore')

In [14]:
df.head(1) # se crean las 3 nuevas columnas

,host_identity_verified,neighbourhood_group,lat,long,instant_bookable,cancellation_policy,room_type,construction_year,price,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day
0,unconfirmed,Brooklyn,40.64749,-73.97237,False,strict,Private room,2020.0,966.0,193.0,10.0,9.0,0.21,4.0,6.0,286.0,2021,10,19


### Quitar caracteres extraños de la columna barrio

In [15]:
# quitar caracteres extraños de la columna barrio
# en esta versión hemos quitado la columna desde el principio
#df['neighbourhood'] = df['neighbourhood'].str.replace('-', '').str.replace('"', '').str.replace("'", '').str.replace(' ', '_').str.replace('*',' ').str.replace('$',' ')

In [16]:
valores_unicos_all(df)

Column host_identity_verified 	 has 2 unique values
Column neighbourhood_group 	 has 5 unique values
Column lat 	 has 21998 unique values
Column long 	 has 17780 unique values
Column instant_bookable 	 has 2 unique values
Column cancellation_policy 	 has 3 unique values
Column room_type 	 has 4 unique values
Column construction_year 	 has 20 unique values
Column price 	 has 1152 unique values
Column service_fee 	 has 232 unique values
Column minimum_nights 	 has 14 unique values
Column number_of_reviews 	 has 476 unique values
Column reviews_per_month 	 has 1016 unique values
Column review_rate_number 	 has 5 unique values
Column calculated_host_listings_count 	 has 78 unique values
Column availability_365 	 has 437 unique values
Column review_year 	 has 11 unique values
Column review_month 	 has 12 unique values
Column review_day 	 has 31 unique values


In [17]:
valores_unicos_cat(df)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
4
________________________________________________________
Valores únicos en la columna 'host_identity_verified':
cantidad de valores únicos: 2
Los valores únicos de de la columna host_identity_verified son: ['unconfirmed' 'verified']

['unconfirmed' 'verified']
usando value_counts: host_identity_verified
unconfirmed    34779
verified       34526
Name: count, dtype: int64

________________________________________________________
Valores únicos en la columna 'neighbourhood_group':
cantidad de valores únicos: 5
Los valores únicos de de la columna neighbourhood_group son: ['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']

['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']
usando value_counts: neighbourhood_group
Manhattan        29587
Brooklyn         27895
Queens            9262
Bronx             1912
Staten Island      649
Name: count, dtype: int64

______________________________________

['host_identity_verified',
 'neighbourhood_group',
 'cancellation_policy',
 'room_type']

### Verificar estructura final del dataset y guardarlo

In [18]:
# Verificar estructura final del dataset
print("\n Dataset después del prepreprocesamiento:")
print(df.info())

# Guardar el dataset preprocesado
df.to_csv("dataset_limpio.csv", index=False)


 Dataset después del prepreprocesamiento:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69305 entries, 0 to 69304
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   host_identity_verified          69305 non-null  object 
 1   neighbourhood_group             69305 non-null  object 
 2   lat                             69305 non-null  float64
 3   long                            69305 non-null  float64
 4   instant_bookable                69305 non-null  bool   
 5   cancellation_policy             69305 non-null  object 
 6   room_type                       69305 non-null  object 
 7   construction_year               69305 non-null  float64
 8   price                           69305 non-null  float64
 9   service_fee                     69305 non-null  float64
 10  minimum_nights                  69305 non-null  float64
 11  number_of_reviews               69305 non-null  fl

In [19]:
categorical_columns = marcar_cat_columns(df)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
4


In [20]:
print(categorical_columns)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']


Columnas categóricas útiles: ['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']


In [21]:
print(numerical_columns)

['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365']


In [22]:
# hacemos un clustermap que agrupe los parecidos
#sns.clustermap(df[numerical_columns], metric='euclidean', cmap='viridis')

In [23]:
'''# Ajuste del tamaño de la figura
plt.figure(figsize=(12, 10))

# Generación del clustermap con mejoras visuales
clustermap = sns.clustermap(df[numerical_columns],
                            metric='euclidean',
                            cmap='viridis',
                            standard_scale=1,  # Estandarizar las columnas
                            linewidths=0.1,  # Ajustar el grosor de las líneas
                            linecolor='gray',  # Ajustar el color de las líneas
                            figsize=(12, 10)) # Ajustar el tamaño del clustermap

# Añadir título descriptivo
clustermap.ax_heatmap.set_title("Clustermap de Variables Numéricas", fontsize=16, y=1.02)

# Ajuste del diseño para evitar recortes
plt.tight_layout()

# Guardar la figura con un nombre descriptivo
plt.savefig("clustermap_variables_numericas.png", bbox_inches='tight')

# Visualización del gráfico
plt.show()'''

'# Ajuste del tamaño de la figura\nplt.figure(figsize=(12, 10))\n\n# Generación del clustermap con mejoras visuales\nclustermap = sns.clustermap(df[numerical_columns],\n                            metric=\'euclidean\',\n                            cmap=\'viridis\',\n                            standard_scale=1,  # Estandarizar las columnas\n                            linewidths=0.1,  # Ajustar el grosor de las líneas\n                            linecolor=\'gray\',  # Ajustar el color de las líneas\n                            figsize=(12, 10)) # Ajustar el tamaño del clustermap\n\n# Añadir título descriptivo\nclustermap.ax_heatmap.set_title("Clustermap de Variables Numéricas", fontsize=16, y=1.02)\n\n# Ajuste del diseño para evitar recortes\nplt.tight_layout()\n\n# Guardar la figura con un nombre descriptivo\nplt.savefig("clustermap_variables_numericas.png", bbox_inches=\'tight\')\n\n# Visualización del gráfico\nplt.show()'

## Preprocesados (20%)

### Pruebo antes de hacer nada

In [24]:
# pruebo a crear el modelo antes, para ver como mejora después
X_num = df[numerical_columns].drop('price', axis=1) 
# OJO solo uso las columnas numericas, salvo la q quiero predecir que es el precio
y_num = df['price']

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.20, random_state=42)

df_resultados = pd.DataFrame(columns=['Modelo', 'Preprocesado', 'R2', 'MAE', 'RMSE', 'MAPE'])

def calculate_metrics(preprocessor_name, X_train_num, X_test_num, y_train_num, y_test_num):
    models = {
        'LinearRegression': LinearRegression(),
        'KNN': KNeighborsRegressor(),
        'SVR': SVR(),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'RandomForest': RandomForestRegressor(random_state=42)
    }
    for model_name, model in models.items():
        model.fit(X_train_num, y_train_num)
        y_pred_num = model.predict(X_test_num) # cada vez que llamo esta funcion se crea una prrediccion, pero no la estoy guardando ni haciendo transform
        df_resultados.loc[len(df_resultados)] = [model_name, preprocessor_name, r2_score(y_test_num, y_pred_num), mean_absolute_error(y_test_num, y_pred_num),root_mean_squared_error(y_test_num, y_pred_num),mean_absolute_percentage_error(y_test_num, y_pred_num)]
    
    return df_resultados.sort_values('R2', ascending=False)

In [25]:
print(f"Conjunto de Train: {X_train_num.shape, y_train_num.shape}")
print(f"Conjunto de Test: {X_test_num.shape, y_test_num.shape}")

Conjunto de Train: ((55444, 11), (55444,))
Conjunto de Test: ((13861, 11), (13861,))


In [26]:
calculate_metrics('Sin preprocesado', X_train_num, X_test_num, y_train_num, y_test_num)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
0,LinearRegression,Sin preprocesado,0.994214,3.048872,25.154230,0.007891
4,RandomForest,Sin preprocesado,0.993059,4.009534,27.551948,0.011149
1,KNN,Sin preprocesado,0.991447,10.843167,30.583611,0.030712
3,DecisionTree,Sin preprocesado,0.988594,4.635861,35.318957,0.013428
2,SVR,Sin preprocesado,0.563997,188.602085,218.365165,0.672745


GridSearch va a tardar mucho, mejor lo hacemos con unos pocos parámetros:

In [27]:
X = df[numerical_columns].drop('price', axis=1)
y = df['price']
# clasificación multiclase
# X = df.drop('room_type', axis=1)
# y = df['room_type']
model = DecisionTreeRegressor(random_state=42)
params = {
    'max_depth': np.arange(1,2),
    'min_samples_split': np.arange(2,3),
    'criterion': ['squared_error', 'absolute_error']
}
grid = GridSearchCV(model, params, scoring=['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_absolute_percentage_error', 'neg_root_mean_squared_error'], refit='neg_mean_absolute_error', verbose=1) # cv=5 por defecto
grid.fit(X, y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': array([1]),
                         'min_samples_split': array([2])},
             refit='neg_mean_absolute_error',
             scoring=['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error',
                      'neg_mean_absolute_percentage_error',
                      'neg_root_mean_squared_error'],
             verbose=1)

##### Vemos que el mejor estimador, dentro de estos rangos tan pequeños, es el de decisionTreeRegresor con max_depth=1, min_samples=2

### 1️⃣ Preprocesamiento de Variables Numéricas

Aplicamos: 

✅ Imputer para rellenar valores nulos.

✅ Scaler para normalizar los valores y mejorar el rendimiento de los modelos.

✅ Transformer para realizar transformaciones adicionales si es necesario.

### 2️⃣ Preprocesamiento de Variables Categóricas

✅ Imputación de 'others' en columnas categóricas.

✅ Codificación con OrdinalEncoder o OneHotEncoder según corresponda.

Variables categóricas como el suburbio (barrio) se podrían imputar con las coordenadas

In [28]:
categorical_columns = marcar_cat_columns(df)
print(categorical_columns)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
4
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']


In [29]:
numerical_columns = marcar_num_columns(df)
print(numerical_columns)

['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day']
15
['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day']


#### Imputer todas

In [30]:
# imputer solo para las numericas
imputer_numerical = SimpleImputer(strategy='constant', fill_value=0)
np_numerical = imputer_numerical.fit_transform(df[numerical_columns]) 

# imputer solo para las categoricas
imputer_categorical = SimpleImputer(strategy='constant', fill_value='others')
np_categorical = imputer_categorical.fit_transform(df[categorical_columns])

# imputer combinado de todo el df
df_imputer = pd.concat(
    [
        pd.DataFrame(np_numerical, columns=numerical_columns).reset_index(drop=True), # categoricas
        pd.DataFrame(np_categorical, columns=categorical_columns).reset_index(drop=True) # numéricas
    ],
    axis=1
)
df_imputer.head(1)

,lat,long,instant_bookable,construction_year,price,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day,host_identity_verified,neighbourhood_group,cancellation_policy,room_type
0,40.64749,-73.97237,0.0,2020.0,966.0,193.0,10.0,9.0,0.21,4.0,6.0,286.0,2021.0,10.0,19.0,unconfirmed,Brooklyn,strict,Private room


In [31]:
# pruebo con todo tipo de columnas ya que he hecho el df_imputer combinado
X_all_imp = df_imputer.drop('price', axis=1) 
y_all_imp = df_imputer['price']

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all_imp, y_all_imp, test_size=0.20, random_state=42)

In [32]:
df.head(1)

,host_identity_verified,neighbourhood_group,lat,long,instant_bookable,cancellation_policy,room_type,construction_year,price,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day
0,unconfirmed,Brooklyn,40.64749,-73.97237,False,strict,Private room,2020.0,966.0,193.0,10.0,9.0,0.21,4.0,6.0,286.0,2021,10,19


In [33]:
# da error
#calculate_metrics('Com imputer num', X_train_all, X_test_all, y_train_all, y_test_all)
# solo para las numericas

ver cual es mejor, si este, minmaxescaler etc segun los valores

**Estandarización**_: `StandardScaler()`

    - Pros:
        - Simple y fácil de implementar.
        - Capta la media y la desviación estandar.
    - Contras:
        - Asume que los datos son normales.
        - Sensible a distribuciones asimétricas y a los outliers.

**Escalado Min Max**_: `MinMaxScaler()`

    - Pros:
        - Simple y fácil de implementar.
        - Escala los datos a un rango específico (entre 0 y 1).
        - Conserva las proporciones entre las observaciones.
    - Contras:
        - Muy sensible a valores atípicos, ya que afectan al rango de los datos.

**Escalado Robusto**_: `RobustScaler()`

    - Pros:
        - Como su nombre indica, es robusto a los valores atípicos, ya que utiliza medianas y cuantiles.
    - Contras:
        - No es la mejor opción cuando la distribución de los datos es simétrica.
        - Puede perder información en los extremos de la distribución.        

In [34]:
print(numerical_columns)
print(categorical_columns)
categorical_columns_util = ['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
print(categorical_columns_util)

['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day']
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']


In [35]:
print(f"añadimos las nuevas columnas 'price_scaled' y 'number_of_reviews_scaled'")

# Escalar la columna 'price'
scaler = RobustScaler()
df['price_scaled'] = scaler.fit_transform(df[['price']])

# Escalar la columna 'number_of_reviews'
df['number_of_reviews_scaled'] = scaler.fit_transform(df[['number_of_reviews']])

añadimos las nuevas columnas 'price_scaled' y 'number_of_reviews_scaled'


In [36]:
df.head(1)

,host_identity_verified,neighbourhood_group,lat,long,instant_bookable,cancellation_policy,room_type,construction_year,price,service_fee,...,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day,price_scaled,number_of_reviews_scaled
0,unconfirmed,Brooklyn,40.64749,-73.97237,False,strict,Private room,2020.0,966.0,193.0,...,9.0,0.21,4.0,6.0,286.0,2021,10,19,0.596615,0.068966


#### StandardScaler

In [37]:
# Aplicamos standardscaler sobre las columnas numericas que teníamos antes OJO no estamos incluyendo las escaladas anteriores
X_num = df[numerical_columns].drop('price', axis=1) 
y_num = df['price']

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.20, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train_num) # fit solo sobre train y no en test para evitar data leakage

X_train_scaled = scaler.transform(X_train_num) # devuelve un array de numpy
X_test_scaled = scaler.transform(X_test_num) # devuelve un array de numpy

# opcional, pasarlo a dataframes de pandas con los nombres de las columnas
X_train_scaled_SS = pd.DataFrame(X_train_scaled, columns=X_num.columns)
X_test_scaled_SS = pd.DataFrame(X_test_scaled, columns=X_num.columns)
X_train_scaled_SS.head(2)

,lat,long,instant_bookable,construction_year,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day
0,1.210919,0.148276,-0.995250,-0.954617,-0.755201,-0.141614,0.551772,0.123734,0.540237,-0.228709,0.764100,-0.160537,-0.260923,1.495201
1,0.088881,-0.182802,1.004773,0.609068,0.392306,-0.831039,-0.537648,-0.302885,-1.848035,-0.228709,1.054222,-0.160537,0.077834,-0.163348


In [38]:
calculate_metrics('StandardScaler', X_train_scaled_SS, X_test_scaled_SS, y_train_num, y_test_num)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
0,LinearRegression,Sin preprocesado,0.994214,3.048872,25.154230,0.007891
5,LinearRegression,StandardScaler,0.994214,3.054690,25.155641,0.007914
4,RandomForest,Sin preprocesado,0.993059,4.009534,27.551948,0.011149
9,RandomForest,StandardScaler,0.992992,4.070367,27.683829,0.010889
1,KNN,Sin preprocesado,0.991447,10.843167,30.583611,0.030712
8,DecisionTree,StandardScaler,0.989041,4.506202,34.620267,0.011291
3,DecisionTree,Sin preprocesado,0.988594,4.635861,35.318957,0.013428
7,SVR,StandardScaler,0.931722,53.450553,86.412864,0.194145
6,KNN,StandardScaler,0.922377,70.820237,92.136916,0.230805
2,SVR,Sin preprocesado,0.563997,188.602085,218.365165,0.672745


#### MinMaxScaler

In [39]:
# aplicamos MinMaxScaler sobre las columnas numericas
# con las columnas numericas iniciales, OJO
X_num = df[numerical_columns].drop('price', axis=1) 
y_num = df['price']

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.20, random_state=42)

scaler = MinMaxScaler()

X_train_scaled_MM = scaler.fit_transform(X_train_num) # equivalente al ejemplo anterior pero todo en una línea
X_test_scaled_MM = scaler.transform(X_test_num)
X_train_scaled_MM = pd.DataFrame(X_train_scaled_MM, columns=X_num.columns)
X_test_scaled_MM = pd.DataFrame(X_test_scaled_MM, columns=X_num.columns)
X_train_scaled_MM.head(2)

,lat,long,instant_bookable,construction_year,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day
0,0.709418,0.566395,0.0,0.210526,0.282609,0.307692,0.055664,0.016669,0.75,0.0,0.610092,0.7,0.363636,0.933333
1,0.559128,0.535621,1.0,0.684211,0.613043,0.076923,0.000000,0.008668,0.00,0.0,0.699541,0.7,0.454545,0.433333


In [40]:
calculate_metrics('MinMaxScaler', X_train_scaled_MM, X_test_scaled_MM, y_train_num, y_test_num)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
0,LinearRegression,Sin preprocesado,0.994214,3.048872,25.154230,0.007891
5,LinearRegression,StandardScaler,0.994214,3.054690,25.155641,0.007914
10,LinearRegression,MinMaxScaler,0.994214,3.054690,25.155641,0.007914
4,RandomForest,Sin preprocesado,0.993059,4.009534,27.551948,0.011149
9,RandomForest,StandardScaler,0.992992,4.070367,27.683829,0.010889
14,RandomForest,MinMaxScaler,0.992978,4.072061,27.712588,0.010864
1,KNN,Sin preprocesado,0.991447,10.843167,30.583611,0.030712
8,DecisionTree,StandardScaler,0.989041,4.506202,34.620267,0.011291
13,DecisionTree,MinMaxScaler,0.988823,4.523662,34.962259,0.011352
3,DecisionTree,Sin preprocesado,0.988594,4.635861,35.318957,0.013428


#### RobustScaler

In [41]:
# aplicamos RobustScaler sobre las columnas numericas iniciales, OJO!!!
X_num = df[numerical_columns].drop('price', axis=1) 
y_num = df['price']

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.20, random_state=42)

scaler = RobustScaler()
X_train_scaled_RS = scaler.fit_transform(X_train_num)
X_test_scaled_RS = scaler.transform(X_test_num)
X_train_scaled_RS = pd.DataFrame(X_train_scaled_RS, columns=X_num.columns)
X_test_scaled_RS = pd.DataFrame(X_test_scaled_RS, columns=X_num.columns)
X_train_scaled_RS.head(2)

,lat,long,instant_bookable,construction_year,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day
0,0.988647,0.249710,0.0,-0.555556,-0.438596,0.25,1.724138,0.503497,0.5,0.0,0.490494,0.0,-0.333333,1.0
1,0.141506,-0.074719,1.0,0.444444,0.228070,-0.50,-0.241379,0.000000,-1.0,0.0,0.638783,0.0,0.000000,0.0


In [42]:
calculate_metrics('RobustScaler', X_train_scaled_RS, X_test_scaled_RS, y_train_num, y_test_num)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
0,LinearRegression,Sin preprocesado,0.994214,3.048872,25.154230,0.007891
15,LinearRegression,RobustScaler,0.994214,3.054690,25.155641,0.007914
10,LinearRegression,MinMaxScaler,0.994214,3.054690,25.155641,0.007914
5,LinearRegression,StandardScaler,0.994214,3.054690,25.155641,0.007914
4,RandomForest,Sin preprocesado,0.993059,4.009534,27.551948,0.011149
19,RandomForest,RobustScaler,0.993000,4.067544,27.669333,0.010865
9,RandomForest,StandardScaler,0.992992,4.070367,27.683829,0.010889
14,RandomForest,MinMaxScaler,0.992978,4.072061,27.712588,0.010864
1,KNN,Sin preprocesado,0.991447,10.843167,30.583611,0.030712
18,DecisionTree,RobustScaler,0.989058,4.499330,34.592810,0.011289


In [43]:
X_train_scaled_RS.skew() # cuanto más cercano a 0 más simétrico

lat                                0.223711
long                               1.257838
instant_bookable                   0.009523
construction_year                  0.002853
service_fee                       -0.000010
minimum_nights                     1.166511
number_of_reviews                  3.922567
reviews_per_month                 10.024844
review_rate_number                -0.138252
calculated_host_listings_count     6.531372
availability_365                   0.357725
review_year                       -0.165815
review_month                       0.240946
review_day                         0.020660
dtype: float64

#### QuantileTransform

Parámetros importantes:

* n_quantiles: Número de cuantiles a calcular. Un valor más alto proporciona una transformación más precisa, pero puede aumentar el tiempo de cálculo.
* output_distribution: Distribución objetivo de la transformación ('uniform' o 'normal').
* random_state: Semilla para el generador de números aleatorios (si se utiliza output_distribution='normal').

Consideraciones:

* QuantileTransformer es una transformación no lineal, por lo que puede distorsionar las relaciones lineales entre las características.
* Si tus datos ya siguen una distribución normal, es posible que QuantileTransformer no mejore el rendimiento y podría incluso empeorarlo.
* Al igual que otros escaladores, QuantileTransformer solo se debe ajustar en el conjunto de datos de entrenamiento y luego aplicarse a los conjuntos de datos de prueba y validación para evitar la fuga de datos.

In [44]:
# aplicamos QuantileTransform a las numericas iniciales, OJO!!
X_num = df[numerical_columns].drop('price', axis=1) 
y_num = df['price']

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.20, random_state=42)

transformer = QuantileTransformer() # pondría valores de n_quantiles y output_distribution pero se puede hacer eterno el nb
X_train_transformed_QT = transformer.fit_transform(X_train_num)
X_test_transformed_QT = transformer.transform(X_test_num)
X_train_transformed_QT = pd.DataFrame(X_train_transformed_QT, columns=X_num.columns)
X_test_transformed_QT = pd.DataFrame(X_test_transformed_QT, columns=X_num.columns)
X_train_transformed_QT.head(2)

,lat,long,instant_bookable,construction_year,service_fee,minimum_nights,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day
0,0.864095,0.673031,0.0,0.226226,0.288288,0.653654,0.845846,0.721722,0.657157,0.0,0.711211,0.461962,0.310310,0.916416
1,0.567091,0.447195,1.0,0.673674,0.621622,0.124124,0.000000,0.499499,0.000000,0.0,0.773273,0.461962,0.523524,0.443944


In [45]:
X_train_transformed_QT.skew() # vemos que se ha recudido la asimetría

lat                               0.001422
long                              0.007361
instant_bookable                  0.009523
construction_year                -0.008272
service_fee                      -0.030604
minimum_nights                    0.246763
number_of_reviews                -0.116415
reviews_per_month                 0.011348
review_rate_number                0.153186
calculated_host_listings_count    0.520797
availability_365                  0.036354
review_year                       0.249765
review_month                     -0.045986
review_day                       -0.020399
dtype: float64

In [46]:
calculate_metrics('QuantileTransformer', X_train_transformed_QT, X_test_transformed_QT, y_train_num, y_test_num)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
0,LinearRegression,Sin preprocesado,0.994214,3.048872,25.154230,0.007891
5,LinearRegression,StandardScaler,0.994214,3.054690,25.155641,0.007914
10,LinearRegression,MinMaxScaler,0.994214,3.054690,25.155641,0.007914
15,LinearRegression,RobustScaler,0.994214,3.054690,25.155641,0.007914
20,LinearRegression,QuantileTransformer,0.993988,5.476580,25.641659,0.011400
4,RandomForest,Sin preprocesado,0.993059,4.009534,27.551948,0.011149
19,RandomForest,RobustScaler,0.993000,4.067544,27.669333,0.010865
9,RandomForest,StandardScaler,0.992992,4.070367,27.683829,0.010889
14,RandomForest,MinMaxScaler,0.992978,4.072061,27.712588,0.010864
24,RandomForest,QuantileTransformer,0.992964,4.073164,27.738810,0.010952


In [47]:
print('skew antes:\n', X_train_scaled_RS.skew())
print('\nskew después:\n', X_train_transformed_QT.skew())

skew antes:
 lat                                0.223711
long                               1.257838
instant_bookable                   0.009523
construction_year                  0.002853
service_fee                       -0.000010
minimum_nights                     1.166511
number_of_reviews                  3.922567
reviews_per_month                 10.024844
review_rate_number                -0.138252
calculated_host_listings_count     6.531372
availability_365                   0.357725
review_year                       -0.165815
review_month                       0.240946
review_day                         0.020660
dtype: float64

skew después:
 lat                               0.001422
long                              0.007361
instant_bookable                  0.009523
construction_year                -0.008272
service_fee                      -0.030604
minimum_nights                    0.246763
number_of_reviews                -0.116415
reviews_per_month                 0.011

In [48]:
print(categorical_columns)

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']


In [49]:
df.keys()

Index(['host_identity_verified', 'neighbourhood_group', 'lat', 'long',
       'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month', 'review_rate_number',
       'calculated_host_listings_count', 'availability_365', 'review_year',
       'review_month', 'review_day', 'price_scaled',
       'number_of_reviews_scaled'],
      dtype='object')

#### OneHotEncoder

In [50]:
# Hay que volver a asignar valores a X y a y porque antes solo tenian los numericos
X_all = df.drop('price', axis=1)
y_all = df['price']

# y hacer el split entre entrenamiento y test
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y_all, test_size=0.20, random_state=42)

# obtener nombres de columnas numéricas y categóricas de X_train_all (sin price)
numerical_columns = marcar_num_columns(X_train_all)
categorical_columns = marcar_cat_columns(X_train_all)

#encoder = OneHotEncoder(sparse_output=False) # sparse_output=False para obtenerlo como matriz de 0s y 1s , probar drop='first'
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # para ignorar cuando haya valores en test que no esten en train
X_train_encoded_OHE = encoder.fit_transform(X_train_all[categorical_columns]) # array de numpy con las codificaciones
X_test_encoded_OHE = encoder.transform(X_test_all[categorical_columns])

''' el OneHotEncoder encontró categorías en el conjunto de prueba (X_test) que no estaban presentes en el conjunto de entrenamiento (X_train) durante el proceso de transformación.'''
'''La forma más común de manejar este problema es configurar el parámetro handle_unknown del OneHotEncoder a 'ignore'. 
Esto le indica al encoder que ignore las categorías desconocidas en el conjunto de prueba y las codifique como columnas de ceros.'''

# pasarlo a dataframes de pandas y juntarlo con las numéricas para obtener resultado como pd.get_dummies
X_train_final_OHE = pd.concat(
    [
        pd.DataFrame(X_train_encoded_OHE, columns=encoder.get_feature_names_out()).reset_index(drop=True), # categoricas
        X_train_all[numerical_columns].reset_index(drop=True) # numéricas
    ],
    axis=1
)
X_test_final_OHE = pd.concat(
    [
        pd.DataFrame(X_test_encoded_OHE, columns=encoder.get_feature_names_out()).reset_index(drop=True), # categoricas
        X_test_all[numerical_columns].reset_index(drop=True) # numéricas
    ],
    axis=1
)

['lat', 'long', 'instant_bookable', 'construction_year', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day', 'price_scaled', 'number_of_reviews_scaled']
16
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
4


In [51]:
print(X_train_all.columns)

Index(['host_identity_verified', 'neighbourhood_group', 'lat', 'long',
       'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month', 'review_rate_number',
       'calculated_host_listings_count', 'availability_365', 'review_year',
       'review_month', 'review_day', 'price_scaled',
       'number_of_reviews_scaled'],
      dtype='object')


In [52]:
print(X_train_encoded_OHE) #tenemos encoding de categoricas, falta incluirlas en el df TODO

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 1. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 1. 0.]]


In [53]:
print(type(X_train_encoded_OHE))

<class 'numpy.ndarray'>


In [54]:
calculate_metrics('OneHotEncoder', X_train_final_OHE, X_test_final_OHE, y_train_all, y_test_all)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
25,LinearRegression,OneHotEncoder,1.000000,2.732060e-13,3.710784e-13,9.830486e-16
28,DecisionTree,OneHotEncoder,1.000000,2.885795e-04,2.402413e-02,7.906720e-07
29,RandomForest,OneHotEncoder,1.000000,9.212900e-04,2.762292e-02,2.153158e-06
0,LinearRegression,Sin preprocesado,0.994214,3.048872e+00,2.515423e+01,7.890901e-03
10,LinearRegression,MinMaxScaler,0.994214,3.054690e+00,2.515564e+01,7.914437e-03
5,LinearRegression,StandardScaler,0.994214,3.054690e+00,2.515564e+01,7.914437e-03
15,LinearRegression,RobustScaler,0.994214,3.054690e+00,2.515564e+01,7.914437e-03
20,LinearRegression,QuantileTransformer,0.993988,5.476580e+00,2.564166e+01,1.140000e-02
4,RandomForest,Sin preprocesado,0.993059,4.009534e+00,2.755195e+01,1.114865e-02
19,RandomForest,RobustScaler,0.993000,4.067544e+00,2.766933e+01,1.086477e-02


#### Combinar OneHotEncoder con MinMaxScaler: ELEGIDO

In [55]:
# Combinar OneHotEncoder con MinMaxScaler
# Hay que volver a asignar valores a X y a y porque antes solo tenian los numericos
X_all = df.drop('price', axis=1)
y_all = df['price']

# y hacer el split entre entrenamiento y test
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y_all, test_size=0.20, random_state=42)

# obtener nombres de columnas numéricas y categóricas de X_train_all (sin price)
numerical_columns = marcar_num_columns(X_train_all)
categorical_columns = marcar_cat_columns(X_train_all)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse_output=False para obtenerlo como matriz de 0s y 1s , probar drop='first'
X_train_encoded_ohe = encoder.fit_transform(X_train_all[categorical_columns]) # array de numpy con las codificaciones
X_test_encoded_ohe = encoder.transform(X_test_all[categorical_columns])

scaler = MinMaxScaler()
X_train_scaled_mms = scaler.fit_transform(X_train_all[numerical_columns])
X_test_scaled_mms = scaler.transform(X_test_all[numerical_columns])

X_train_final_ohe_mms = pd.concat(
    [
        pd.DataFrame(X_train_encoded_ohe, columns=encoder.get_feature_names_out()).reset_index(drop=True), # categoricas
        pd.DataFrame(X_train_scaled_mms, columns=numerical_columns).reset_index(drop=True) # numéricas
    ],
    axis=1
)
X_test_final_ohe_mms = pd.concat(
    [
        pd.DataFrame(X_test_encoded_ohe, columns=encoder.get_feature_names_out()).reset_index(drop=True), # categoricas
        pd.DataFrame(X_test_scaled_mms, columns=numerical_columns).reset_index(drop=True) # numéricas
    ],
    axis=1
)

# Voy a elegir este preprocesado! X_train_final_ohe_mms

['lat', 'long', 'instant_bookable', 'construction_year', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day', 'price_scaled', 'number_of_reviews_scaled']
16
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
4


In [56]:
X_train_final_ohe_mms.head(1)

,host_identity_verified_unconfirmed,host_identity_verified_verified,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,...,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day,price_scaled,number_of_reviews_scaled
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.055664,0.016669,0.75,0.0,0.610092,0.7,0.363636,0.933333,0.284348,0.055664


In [57]:
X_train_final_ohe_mms.columns

Index(['host_identity_verified_unconfirmed', 'host_identity_verified_verified',
       'neighbourhood_group_Bronx', 'neighbourhood_group_Brooklyn',
       'neighbourhood_group_Manhattan', 'neighbourhood_group_Queens',
       'neighbourhood_group_Staten Island', 'cancellation_policy_flexible',
       'cancellation_policy_moderate', 'cancellation_policy_strict',
       'room_type_Entire home/apt', 'room_type_Hotel room',
       'room_type_Private room', 'room_type_Shared room', 'lat', 'long',
       'instant_bookable', 'construction_year', 'service_fee',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'review_year', 'review_month', 'review_day',
       'price_scaled', 'number_of_reviews_scaled'],
      dtype='object')

In [58]:
calculate_metrics('OneHotEncoder+MinMaxScaler', X_train_final_ohe_mms, X_test_final_ohe_mms, y_train_all, y_test_all)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
30,LinearRegression,OneHotEncoder+MinMaxScaler,1.000000,4.836730e-13,5.938713e-13,1.803472e-15
25,LinearRegression,OneHotEncoder,1.000000,2.732060e-13,3.710784e-13,9.830486e-16
33,DecisionTree,OneHotEncoder+MinMaxScaler,1.000000,2.885795e-04,2.402413e-02,7.906720e-07
28,DecisionTree,OneHotEncoder,1.000000,2.885795e-04,2.402413e-02,7.906720e-07
34,RandomForest,OneHotEncoder+MinMaxScaler,1.000000,9.212900e-04,2.762292e-02,2.153158e-06
29,RandomForest,OneHotEncoder,1.000000,9.212900e-04,2.762292e-02,2.153158e-06
0,LinearRegression,Sin preprocesado,0.994214,3.048872e+00,2.515423e+01,7.890901e-03
15,LinearRegression,RobustScaler,0.994214,3.054690e+00,2.515564e+01,7.914437e-03
10,LinearRegression,MinMaxScaler,0.994214,3.054690e+00,2.515564e+01,7.914437e-03
5,LinearRegression,StandardScaler,0.994214,3.054690e+00,2.515564e+01,7.914437e-03


### Con pipeline

In [59]:
#Tenemos un columntransfer dentro del pipeline, que no admite arrays ni dataframe.
#por eso tengo que pasarle los numeros de las columnas
numerical_columns = marcar_num_columns(df)

X_num_pip = df[numerical_columns].drop('price', axis=1)
y_num_pip = df['price']

numerical_columns_sin_price = marcar_num_columns(X_num_pip)

# Dividir en conjuntos de entrenamiento y prueba
X_train_num_pip, X_test_num_pip, y_train_num_pip, y_test_num_pip = train_test_split(
    X_num_pip, y_num_pip, test_size=0.20, random_state=42
)

# Obtener índices numéricos de las columnas
column_indices = [X_num_pip.columns.get_loc(col) for col in numerical_columns_sin_price]

# Preprocesamiento numérico
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', ColumnTransformer([
        ('std', StandardScaler(), [column_indices[0], column_indices[1], column_indices[2], column_indices[5], column_indices[6], column_indices[7], column_indices[9], column_indices[10], column_indices[11], column_indices[12], column_indices[13], column_indices[14], column_indices[15]]), #Indices de las columnas que se escalan con standardscaler
        ('minmax', MinMaxScaler(), [column_indices[3], column_indices[4]]), #Indices de las columnas que se escalan con minmax
        ('quantile', QuantileTransformer(output_distribution='normal'), [column_indices[8]]) #Indice de la columna que se escala con QuantileTransformer
    ], remainder='passthrough'))
])

# Ajuste y transformación en el conjunto de entrenamiento
X_train_pipeline_num = numeric_pipeline.fit_transform(X_train_num_pip)

# Transformación en el conjunto de prueba
X_test_pipeline_num = numeric_pipeline.transform(X_test_num_pip)

# Imprimir nombres de columnas procesadas y escaladores en formato de tabla
print("Columnas procesadas:")
print("{:<10} {:<35} {:<20}".format("Índice", "Nombre de columna", "Escalador"))  # Encabezados
print("----------------------------------------------------------------")

for i, col_name in enumerate(numerical_columns_sin_price):
    if i in [0, 1, 2, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]:
        scaler_name = "StandardScaler"
    elif i in [3, 4]:
        scaler_name = "MinMaxScaler"
    elif i == 8:
        scaler_name = "QuantileTransformer"
    else:
        scaler_name = "No se aplicó escalador"
    print("{:<10} {:<35} {:<20}".format(column_indices[i], col_name, scaler_name))

# Convertir a DataFrame (opcional)
X_train_pipeline_num = pd.DataFrame(X_train_pipeline_num, columns=numerical_columns_sin_price)
X_test_pipeline_num = pd.DataFrame(X_test_pipeline_num, columns=numerical_columns_sin_price)

print("----------------------------------------------------------------")
print(f"\nAsí queda X_train después de pasarle el pipeline:")
print(X_train_pipeline_num.head())
print("----------------------------------------------------------------")
print(f"\nAsí queda X_test después de pasarle el pipeline:")
print(X_test_pipeline_num.head())

['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day', 'price_scaled', 'number_of_reviews_scaled']
17
['lat', 'long', 'instant_bookable', 'construction_year', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day', 'price_scaled', 'number_of_reviews_scaled']
16
Columnas procesadas:
Índice     Nombre de columna                   Escalador           
----------------------------------------------------------------
0          lat                                 StandardScaler      
1          long                                StandardScaler      
2          instant_bookable                    StandardScaler      
3          construction_year     

In [60]:
# pipeline para columnas categóricas
categorical_columns = marcar_cat_columns(df)

# Verificar que categorical_columns es una lista de strings
if not all(isinstance(col, str) for col in categorical_columns):
    raise ValueError("categorical_columns debe ser una lista de strings.")

# Definir la columna ordinal (si existe)
ordinal_col = 'columna_ordinal' if 'columna_ordinal' in categorical_columns else None

# Seleccionar las columnas categóricas para OneHotEncoding
onehot_cols = [col for col in categorical_columns if col != ordinal_col]

categoric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación de valores faltantes
    ('encoder', ColumnTransformer([
        ('ordinal', OrdinalEncoder(categories=[['valor1', 'valor2', 'valor3']]), ['columna_ordinal']), # Ejemplo: Codificación ordinal
        ('onehot', OneHotEncoder(handle_unknown='ignore'), onehot_cols) # Corrección: onehot_cols en lugar de df[categorical_columns].drop(...)
#        ('onehot', OneHotEncoder(handle_unknown='ignore'), df(categorical_columns).drop('columna_ordinal')) # Ejemplo: OneHotEncoding (se excluye la ordinal)
    ], remainder='passthrough')) #Las que no se especifiquen se mantienen sin escalar
])

['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']
4


In [61]:
print(numerical_columns_sin_price)
print(len(numerical_columns_sin_price))

['lat', 'long', 'instant_bookable', 'construction_year', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day', 'price_scaled', 'number_of_reviews_scaled']
16


In [62]:
df_transformed_pip_num = pd.DataFrame(numeric_pipeline.fit_transform(df[numerical_columns_sin_price]), columns=numeric_pipeline.get_feature_names_out())  # Aplicar solo a numéricas

In [63]:
df_transformed_pip_num.head(1) # este es bueno pero solo para numericas

,std__lat,std__long,std__instant_bookable,std__minimum_nights,std__number_of_reviews,std__reviews_per_month,std__calculated_host_listings_count,std__availability_365,std__review_year,std__review_month,std__review_day,std__price_scaled,std__number_of_reviews_scaled,minmax__construction_year,minmax__service_fee,quantile__review_rate_number
0,-1.439649,-0.462349,-0.993829,1.23476,-0.365215,-0.657858,-0.085519,0.988062,1.047381,1.430786,0.389278,1.030521,-0.365215,0.894737,0.795652,0.384375


### Ahora que tienes tus datos `X_train_pipeline_num` y `X_test_pipeline_num` procesados con el pipeline, el siguiente paso lógico es construir y entrenar un modelo de machine learning. Aquí te presento los pasos a seguir:

**1. Selección del modelo:**

* **Regresión:** Dado que estás intentando predecir el precio, un valor numérico continuo, necesitas un modelo de regresión. Algunas opciones populares son:
    * Regresión Lineal
    * Random Forest Regressor
    * Gradient Boosting Regressor (por ejemplo, XGBoost, LightGBM)
    * Support Vector Regression (SVR)
    
* **Consideraciones:** La elección del modelo depende de varios factores, como la complejidad de tus datos, la interpretabilidad deseada y el rendimiento esperado.

**2. Entrenamiento del modelo:**

* **Ajuste del modelo:** Utiliza el método `fit()` del modelo seleccionado para entrenarlo con tus datos de entrenamiento `X_train_pipeline_num` y la variable objetivo `y_train_num_pip`.
* **Ejemplo (Random Forest Regressor):**

In [64]:
model_RF = RandomForestRegressor(random_state=42)
model_RF.fit(X_train_pipeline_num, y_train_num_pip)

RandomForestRegressor(random_state=42)

In [65]:
calculate_metrics('RandomForest after pipeline', X_train_pipeline_num, X_test_pipeline_num, y_train_num_pip, y_test_num_pip)

,Modelo,Preprocesado,R2,MAE,RMSE,MAPE
35,LinearRegression,RandomForest after pipeline,1.000000,6.702487e-13,7.819360e-13,2.160234e-15
25,LinearRegression,OneHotEncoder,1.000000,2.732060e-13,3.710784e-13,9.830486e-16
30,LinearRegression,OneHotEncoder+MinMaxScaler,1.000000,4.836730e-13,5.938713e-13,1.803472e-15
28,DecisionTree,OneHotEncoder,1.000000,2.885795e-04,2.402413e-02,7.906720e-07
33,DecisionTree,OneHotEncoder+MinMaxScaler,1.000000,2.885795e-04,2.402413e-02,7.906720e-07
39,RandomForest,RandomForest after pipeline,1.000000,9.147969e-04,2.649416e-02,2.119070e-06
34,RandomForest,OneHotEncoder+MinMaxScaler,1.000000,9.212900e-04,2.762292e-02,2.153158e-06
29,RandomForest,OneHotEncoder,1.000000,9.212900e-04,2.762292e-02,2.153158e-06
38,DecisionTree,RandomForest after pipeline,1.000000,3.607243e-04,3.062488e-02,5.782975e-07
0,LinearRegression,Sin preprocesado,0.994214,3.048872e+00,2.515423e+01,7.890901e-03


In [66]:
#Hacemos una predicción del precio con el modelo que hemos entrenado
y_pred = model_RF.predict(X_test_pipeline_num)

r2 = r2_score(y_test_num_pip, y_pred)
mse = mean_squared_error(y_test_num_pip, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_num_pip, y_pred)
mape = mean_absolute_percentage_error(y_test_num_pip, y_pred)

print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.4f}")

R²: 1.0000
MSE: 0.0007
RMSE: 0.0265
MAE: 0.0009
MAPE: 0.0000


In [67]:
print(numeric_pipeline)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('std', StandardScaler(),
                                                  [0, 1, 2, 5, 6, 7, 9, 10, 11,
                                                   12, 13, 14, 15]),
                                                 ('minmax', MinMaxScaler(),
                                                  [3, 4]),
                                                 ('quantile',
                                                  QuantileTransformer(output_distribution='normal'),
                                                  [8])]))])


In [68]:
#print(categoric_pipeline)

In [69]:
print(numerical_columns)
print(categorical_columns)

['lat', 'long', 'instant_bookable', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'review_year', 'review_month', 'review_day', 'price_scaled', 'number_of_reviews_scaled']
['host_identity_verified', 'neighbourhood_group', 'cancellation_policy', 'room_type']


In [70]:
# Transformación de datos
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numerical_columns),
    ('cat', categoric_pipeline, categorical_columns)
])

In [71]:
#  Aplicación del preprocesamiento
#df_transformed = pd.DataFrame(preprocessor.fit_transform(df), columns=preprocessor.get_feature_names_out())

In [72]:
df_transformed_pip_num.head(1)

,std__lat,std__long,std__instant_bookable,std__minimum_nights,std__number_of_reviews,std__reviews_per_month,std__calculated_host_listings_count,std__availability_365,std__review_year,std__review_month,std__review_day,std__price_scaled,std__number_of_reviews_scaled,minmax__construction_year,minmax__service_fee,quantile__review_rate_number
0,-1.439649,-0.462349,-0.993829,1.23476,-0.365215,-0.657858,-0.085519,0.988062,1.047381,1.430786,0.389278,1.030521,-0.365215,0.894737,0.795652,0.384375


### 3️⃣ Resumen del Preprocesamiento

✅ Columnas Numéricas: Se ha aplicado imputación y escalado con SimpleImputer y StandardScaler.

✅ Columnas Categóricas:

    * OrdinalEncoder para room_type. COMPROBAR
    * OneHotEncoder para neighbourhood_group y neighbourhood. COMPROBAR TO DO

Ahora los datos están listos para la siguiente fase de análisis y modelado! 

*Aquí ya tengo que tener el dataframe listo!!! o la matriz X_train X_test*

### Finalmente me quedo con el resultado de usar OneHotEncoder, y MinMaxScaler

X_train_final_ohe_mms

X_test_final_ohe_mms

y_train_all

y_test_all

In [73]:
print(type(X_train_final_ohe_mms))
print(type(X_test_final_ohe_mms))
print(type(y_train_all))
print(type(y_test_all))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [74]:
X_train_final_ohe_mms.head()

,host_identity_verified_unconfirmed,host_identity_verified_verified,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,...,number_of_reviews,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,review_year,review_month,review_day,price_scaled,number_of_reviews_scaled
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.055664,0.016669,0.75,0.000000,0.610092,0.7,0.363636,0.933333,0.284348,0.055664
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,0.008668,0.00,0.000000,0.699541,0.7,0.454545,0.433333,0.613043,0.000000
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000977,0.001667,1.00,0.003021,0.082569,0.6,1.000000,0.933333,0.615652,0.000977
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.001953,0.000667,0.50,0.012085,0.860092,0.8,1.000000,0.100000,0.993913,0.001953
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.011719,0.006779,0.25,0.000000,0.022936,0.7,0.454545,0.333333,0.377391,0.011719


In [75]:
# Asumiendo que tienes X_train_final_ohe_mms, X_test_final_ohe_mms,
# y_train_all e y_test_all como arrays de NumPy o Series de pandas

# 1. Convertir a DataFrames y Series
X_train_df = pd.DataFrame(X_train_final_ohe_mms)
X_test_df = pd.DataFrame(X_test_final_ohe_mms)

if isinstance(y_train_all, np.ndarray):
    y_train_series = pd.Series(y_train_all)
else:
    y_train_series = y_train_all

if isinstance(y_test_all, np.ndarray):
    y_test_series = pd.Series(y_test_all)
else:
    y_test_series = y_test_all

# 2. Añadir la variable objetivo
X_train_df['target'] = y_train_series
X_test_df['target'] = y_test_series

# 3. Concatenar los DataFrames
df_final = pd.concat([X_train_df, X_test_df], ignore_index=True)

# Imprimir las primeras filas del DataFrame resultante
print(df_final.head())

   host_identity_verified_unconfirmed  host_identity_verified_verified  \
0                                 1.0                              0.0   
1                                 0.0                              1.0   
2                                 1.0                              0.0   
3                                 1.0                              0.0   
4                                 1.0                              0.0   

   neighbourhood_group_Bronx  neighbourhood_group_Brooklyn  \
0                        0.0                           0.0   
1                        0.0                           1.0   
2                        0.0                           0.0   
3                        0.0                           0.0   
4                        0.0                           1.0   

   neighbourhood_group_Manhattan  neighbourhood_group_Queens  \
0                            1.0                         0.0   
1                            0.0                      

In [76]:
print('Nos quedamos con el dataframe df_final, con OneHotEncoder y MinMaxScaler')
print(df_final.head(1))

Nos quedamos con el dataframe df_final, con OneHotEncoder y MinMaxScaler
   host_identity_verified_unconfirmed  host_identity_verified_verified  \
0                                 1.0                              0.0   

   neighbourhood_group_Bronx  neighbourhood_group_Brooklyn  \
0                        0.0                           0.0   

   neighbourhood_group_Manhattan  neighbourhood_group_Queens  \
0                            1.0                         0.0   

   neighbourhood_group_Staten Island  cancellation_policy_flexible  \
0                                0.0                           0.0   

   cancellation_policy_moderate  cancellation_policy_strict  ...  \
0                           0.0                         1.0  ...   

   reviews_per_month  review_rate_number  calculated_host_listings_count  \
0           0.016669                0.75                             0.0   

   availability_365  review_year  review_month  review_day  price_scaled  \
0          0.6100